Begin Sep 5 2019  
Author: Rhondene Wint 
# Find the fraction of introns that can host a tRNA and compare to intergenic

- Perform a two-sided and one-sided binomial test between intronic tRNAs and intergenic tRNAs. Explicitly, What is the probability that 130/288 tRNA genes are nested within protein-coding genes by random chance

#### bear in mind
- Intron length in Dmel ranges from 44bp to >70kbp with a strong mode at 58bp. https://pdfs.semanticscholar.org/fe6a/a335e906d93d28a739b854014638b7cfa83e.pdf
- The majority (54%) of introns in Drosophila melanogaster is not longer than 81 nucleotides (nt). https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2704441/
- About 10% of introns in humans and 5% in Drosophila are >10 kb. An intron in the dyenin of the Y-chromosome of Dmel is 3Mb long (https://www.genetics.org/content/170/2/661)

In [ ]:
from interval import interval, inf, imath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn 

In [ ]:
## load intron bed files
"""all_introns.bed is Flybase r6.22, 
host_introns.bed is a subset of all_introns.bed" containing introns that contain tRNA regions"""
host_introns = pd.read_table('host_introns_only.bed',sep='\t') ##custom bed wirh host intron IDs
all_introns = pd.read_table('dmel_introns_ensembl.bed', sep='\t',header=None, names=['Chr','Start','End','ID', 'Phase', 'Strand'])


In [ ]:
##split columns to get transcript IDs 
IDs = []
for name in all_introns['ID'].values:
    n = name.split('_')[0]
    IDs.append(n)
all_introns['Transcript_ID']=IDs

### compute intron lengths
all_introns['Length']= all_introns['End']- all_introns['Start']
all_introns.head()

all_introns['Length'].describe()
### look at the smallest host intron
host_introns.head()

In [ ]:
host_introns['Length'].describe()

In [ ]:
##plot distirbutions of host intron length
host_introns['Length'].hist()

## to get total introns, I will  merge all overlapping introns in the bed file of a transcript then map the transcript ID to the gene IDs

In [ ]:
transc_ids = all_introns['Transcript_ID'].unique()
transcripts = all_introns.groupby('Transcript_ID')

"""merge overlapping introons for each gene and compute length"""
merged_regions = dict()
## iterate over each gene
for ID in transc_ids:
    df = transcripts.get_group(ID).sort_values(by='Start', ascending=True) ##sort by ascending order
    #set the initial value to start of the earliest intron
    consol_region = interval[df['Start'].values[0]] 
    ##iterate over exons of the gene
    for i in range(df.shape[0]):
        #create an interval of an individual exon region
        intron_size = interval[df['Start'].values[i],df['End'].values[i]] 
        ##consolidate overlapping the intron region
        consol_region= consol_region | intron_size  
    ##finally store  a list of non-overlapping intronic intervals of a gene
    merged_regions[ID]=consol_region

##store total_intron_size for each transcript
transc_total_introns=dict()
for ID in transc_ids:
    consol_region = merged_regions[ID]
    total=0
    for region in consol_region:
        total+= region[1]-region[0]
    
    transc_total_introns[ID] = total  ##for my own use when I want to look intron sise distbtion

In [ ]:
fb_gtf = pd.read_table('dmel-all-r6.27.gtf',sep='\t',header=None, 
                    names=['Chr','Database','Feature','Start','End','?', 'Strand','?.1', 'Attributes'])
fb_gtf.head()

In [ ]:
## split column to transcript_ID infromation 
fb_gtf['Attributes'].values[1]

In [ ]:
##store the total intron lenghts for transcript for each gene in a table
total_introns_transc = pd.DataFrame.from_dict(transc_total_introns,orient='index').reset_index()
total_introns_transc.columns=['Transcript ID', 'Total Intron Size']

In [ ]:
##select transcript entries for mRNA and ncRNA fromt the gtf file
all_transc= fb_gtf.query('Feature!="gene" and Feature!="exon" and Feature!= "5UTR" and Feature!="3UTR" and Feature!="stop_codon" and Feature!="start_codon" and Feature!="CDS"')

In [ ]:
##parse Attributes  to obtain transcript IDs
trans_id = []
for attr in all_transc['Attributes'].values:
    ID = attr.split(";")[2].split(" ")[2].replace('"',"")
    trans_id.append(ID)
all_transc['Transcript ID'] =  trans_id

In [ ]:
## make a dictionary with gene ID: trasncripts
genes = all_transc.groupby('Gene ID')
gene_ids = all_transc['Gene ID'].unique()

gene_dict = dict()
for gene in gene_ids:
    df = genes.get_group(gene)
    transcripts =  list(df['Transcript ID'].values)
    gene_dict[gene]=transcripts
    
""" the map trancsript IDs to parent gene IDs"""
parent_gene = []
for ID in total_introns_transc['Transcript ID']:
    for gene in list(gene_dict.keys()):
        if ID in gene_dict[gene]:
            parent_gene.append(gene)
            break
        else:
            continue    

In [ ]:
len(parent_gene)==total_introns_transc.shape[0]  #should be True but is false 
total_introns_transc.shape[0] - len(parent_gene) ## 

#### Okay so 83 transcript ID in the ensembl annotation based on  Flybase release 6.22 is missing in the flybase release 6.27 gtf.
those 83 gonna get dropped

In [ ]:
""" identify missing transcript entries """
present = []
for i in range(total_introns_transc.shape[0]):
    transc_id = total_introns_transc['Transcript ID'].values[i]
    if transc_id not in all_transc['Transcript ID'].values:
        present.append('Missing')
    else:
        present.append('Yes')
total_introns_transc['Present']=present

""" select entries that are present in both annotation, i.e. filter out the 83 transcitps"""
total_introns2 = total_introns_transc.query('Present=="Yes"')

"""update the code for mapping transcript ID to gen ID"""
parent_gene = []
for ID in total_introns2['Transcript ID'].values:
    for gene in list(gene_dict.keys()):
        if ID in gene_dict[gene]:
            parent_gene.append(gene)
            break
        else:
            continue


In [ ]:
""" compute total intron size"""
parent_genes = total_introns2['Gene ID'].unique()
total_intron_size = 0
genes =  total_introns2.groupby('Gene ID')

for gene in parent_genes:
    ##get all transcripts for each gene
    df = genes.get_group(gene)
    ##update the total genomic intron with the max intron length
    total_intron_size+=df['Total Intron Size'].max()
total_intron_size

## compute net intronic size by subtracting contribution due nestred tRNAs

In [ ]:
##load the trna genes predicted from introns of the longest isoforms
tscan_intronic = pd.read_table('tscan_predicted_transc_introns.txt',sep='\t',skiprows=1).reset_index(drop=True).drop(0,axis=0)
tscan_intronic.head()

In [ ]:
tscan_intronic=tscan_intronic[tscan_intronic['Note']!='pseudo'].drop('Note',axis=1)
tscan_intronic['Score'] = tscan_intronic['Score'].astype(float)
tscan_intronic = tscan_intronic[tscan_intronic['Score']>50]  ##remove low-quality trnas with covariance below 50
lengths = tscan_intronic['End'].astype(int) - tscan_intronic['Begin'].astype(int)
total_trna = np.sum(np.abs(lengths))
total_trna

In [ ]:
#compute net intronic region 
net_intronic = total_intron_size-  total_trna
net_intronic

#### recall that by summing the overlapping exons then subtracting it from total genic composition I got total intron size of 65 Mb (see merge_overlapping_exons notebook) and summing the total introns of the longest isoform in Graham's table gave me 57Mb.

## compute genome size and intergenic size
- Intergenic = genome_size - total geneic

In [1]:
##total genic
genic = fb_gtf.query('Feature=="gene"')  ##keep in mind tRNAs are "gene" too
genic_size = np.sum(genic['End']-genic['Start'])
genic_size

In [ ]:
genome_size=0
with open('dmel-all-chromosome-r6.27.fasta', 'r') as f:
    for line in f:
        if line.startswith('>') is True:
            continue
        else:
            genome_size+=len(line)
genome_size
""" estimate intergnic as difference between genome size and total genic """
intergenic = genome_size-genic_size
print('Total intronic size {}Mb \n Total Intergenic size: {}Mb '.format(total_intron_size/1e6, intergenic))
print("Log Intergenic size: {} \n Log Intronic size {}".format(np.log(intergenic), np.log(total_intron_size)))

## Compute binomial test

In [2]:
from scipy.stats import chisquare, binom_test

In [ ]:
net_intronic/genome_size

In [ ]:
#since total intron size is 46% of genome what is the expectation of nesting 130 trna? 
binom_test(x=130,n=290,p=0.46,alternative='two-sided' )   

### Repeat test by choosing only total intron size based on introns atleast 200nt

In [ ]:
above_200 = all_introns[all_introns['Length']>=200]
above_200.shape
##merge overlapping introns of transcripts
transc_ids = above_200['Transcript_ID'].unique()
transcripts = above_200.groupby('Transcript_ID')

"""merge overlapping introons for each gene and compute length"""
merged_regions = dict()
## iterate over each gene
for ID in transc_ids:
    df = transcripts.get_group(ID).sort_values(by='Start', ascending=True) ##sort by ascending order
    #set the initial value to start of the earliest intron
    consol_region = interval[df['Start'].values[0]] 
    ##iterate over exons of the gene
    for i in range(df.shape[0]):
        #create an interval of an individual intronicregion
        intron_size = interval[df['Start'].values[i],df['End'].values[i]] 
        ##consolidate overlapping the intron region
        consol_region= consol_region | intron_size  
    ##finally store  a list of non-overlapping intronicc intervals of a gene
    merged_regions[ID]=consol_region

##computed total_intron_size for drosophila
transc_total_introns=dict()
total_intron_length=0
for ID in transc_ids:
    consol_region = merged_regions[ID]
    total=0
    for region in consol_region:
        total+= region[1]-region[0]
    
##computed total_intron_size for drosophila
transc_total_introns=dict()
for ID in transc_ids:
    consol_region = merged_regions[ID]
    total=0
    for region in consol_region:
        total+= region[1]-region[0]
    
    transc_total_introns[ID] = total  ##for my own use when I want to look intron sise distbtion
above_200_totals = pd.DataFrame.from_dict(transc_total_introns,orient='index').reset_index()
above_200_totals.columns=['Transcript ID', 'Total Intron Size']

#### aaah i just remembered I can use flybase feature mapper to map transcript ID to gene ID but not gonna. I'll maintiain the same approach

In [ ]:
present = []
for i in range(above_200_totals.shape[0]):
    transc_id = above_200_totals['Transcript ID'].values[i]
    if transc_id not in all_transc['Transcript ID'].values:
        present.append('Missing')
    else:
        present.append('Yes')
above_200_totals['Present']=present  ##34 transcripts will be absent from calculations

above_200_totals2= above_200_totals.query('Present=="Yes"')

"""mapping transcript ID to gen ID"""
parent_gene = []
for ID in above_200_totals2['Transcript ID'].values:
    for gene in list(gene_dict.keys()):
        if ID in gene_dict[gene]:
            parent_gene.append(gene)
            break
        else:
            continue
above_200_totals2['Gene ID']= parent_gene
above_200_totals2.to_csv('introns_above_200nt.csv',index=False, sep=',')
parent_genes = above_200_totals2['Gene ID'].unique()
total_intron_size = 0
genes =  above_200_totals2.groupby('Gene ID')

for gene in parent_genes:
    ##get all transcripts for each gene
    df = genes.get_group(gene)
    ##update the total genomic intron with the max intron length
    total_intron_size+=df['Total Intron Size'].max()
total_intron_size

In [ ]:
net_intronic/genome_size
## in intronic is 
binom_test(x=130,n=290,p=0.46,alternative='two-sided')   
## in intronic is 
binom_test(x=130,n=290,p=0.46,alternative='greater')   

## 5 tRNAS were in the 3'UTR

In [ ]:
##total size of 3UTR trnas
trnas_UTR = np.sum(np.abs(tscan_3UTR['End '].astype(int)- tscan_3UTR['Begin'].astype(int)))
trnas_UTR
total_3UTR/genome_size  ##5% of genome is 3UTR
binom_test(x=130,n=290,p=0.46,alternative='two-sided')   

len(fb_gtf[fb_gtf['Feature']=='mRNA']['Gene ID'].unique())
len(fb_gtf[fb_gtf['Feature']=='3UTR']['Gene ID'].unique())
len(fb_gtf[fb_gtf['Feature']=='mRNA']['Gene ID'].unique())-len(fb_gtf[fb_gtf['Feature']=='3UTR']['Gene ID'].unique()) 
##445 3'utr missing

UTR_3= fb_gtf[fb_gtf['Feature']=='3UTR']

UTR_3['Length']= UTR_3['End']-UTR_3['Start']
UTR_3['Length'].describe()

total_3UTR/genome_size  ##5% of genome is 3UTR

binom_test(x=5,n=290,p=0.06,alternative='two-sided')   ##p-val still == 0.00073
binom_test(x=5,n=290,p=0.06,alternative='two-sided')   ##p-val still < 0.05


### In conclusion:
- Nested intronic tRNAs do not occur more than random
-   3'UTR trnas occur more than expected random. 